# TPCH queries

In [3]:
from pymongo import MongoClient
from bson.son import SON
import pprint
import time
client = MongoClient(port=27017)

In [4]:
db = client['test']
collection = db['orders']

## TPCH query no 1

In [9]:
pipeline1 = [{  
      "$match":{  
         "O_LINEITEMS":{  
            "$elemMatch":{  
               "L_SHIPDATE":{  
                  "$lte": "2000-01-01"
               }
            }
         }
      }
   },
   {  
      "$unwind":"$O_LINEITEMS"
   },
   {  
      "$project":{  
         "O_LINEITEMS.L_RETURNFLAG":1,
         "O_LINEITEMS.L_LINESTATUS":1,
         "O_LINEITEMS.L_QUANTITY":1,
         "O_LINEITEMS.L_EXTENDEDPRICE":1,
         "O_LINEITEMS.L_DISCOUNT":1,
         "l_dis_min_1":{  
            "$subtract":[  
               1,
               "$O_LINEITEMS.L_DISCOUNT"
            ]
         },
         "l_tax_plus_1":{  
            "$add":[  
               "$O_LINEITEMS.L_TAX",
               1
            ]
         }
      }
   },
   {  
      "$group":{  
         "_id":{  
            "L_RETURNFLAG":"$O_LINEITEMS.L_RETURNFLAG",
            "L_LINESTATUS":"$O_LINEITEMS.L_LINESTATUS"
         },
         "sum_qty":{  
            "$sum":"$O_LINEITEMS.L_QUANTITY"
         },
         "sum_base_price":{  
            "$sum":"$O_LINEITEMS.L_EXTENDEDPRICE"
         },
         "sum_disc_price":{  
            "$sum":{  
               "$multiply":[  
                  "$O_LINEITEMS.L_EXTENDEDPRICE",
                  "$l_dis_min_1"
               ]
            }
         },
         "sum_charge":{  
            "$sum":{  
               "$multiply":[  
                  "$O_LINEITEMS.L_EXTENDEDPRICE",
                  {  
                     "$multiply":[  
                        "$l_tax_plus_1",
                        "$l_dis_min_1"
                     ]
                  }
               ]
            }
         },
         "avg_price":{  
            "$avg":"$O_LINEITEMS.L_EXTENDEDPRICE"
         },
         "avg_disc":{  
            "$avg":"$O_LINEITEMSL_.DISCOUNT"
         },
         "count_order":{  
            "$sum":1
         }
      }
   },
   {  
      "$sort":{  
         "O_LINEITEMS.L_RETURNFLAG":1,
         "O_LINEITEMS.L_LINESTATUS":1
      }
   }]

In [10]:
tick = time.time()
pprint.pprint(list(db.orders.aggregate(pipeline1)))

tock = time.time()

print("Total execution time: %.3f s" % (tock - tick))

[{'_id': {'L_LINESTATUS': 'O', 'L_RETURNFLAG': 'N'},
  'avg_disc': None,
  'avg_price': 35703.76059436254,
  'count_order': 30049,
  'sum_base_price': 1072862302.1,
  'sum_charge': 1060424708.624603,
  'sum_disc_price': 1019517788.9930999,
  'sum_qty': 765251},
 {'_id': {'L_LINESTATUS': 'F', 'L_RETURNFLAG': 'R'},
  'avg_disc': None,
  'avg_price': 35874.00653268018,
  'count_order': 14902,
  'sum_base_price': 534594445.35,
  'sum_charge': 528524219.358903,
  'sum_disc_price': 507996454.4067,
  'sum_qty': 381449},
 {'_id': {'L_LINESTATUS': 'F', 'L_RETURNFLAG': 'A'},
  'avg_disc': None,
  'avg_price': 35785.709306937344,
  'count_order': 14876,
  'sum_base_price': 532348211.65,
  'sum_charge': 526165934.000839,
  'sum_disc_price': 505822441.4861,
  'sum_qty': 380456},
 {'_id': {'L_LINESTATUS': 'F', 'L_RETURNFLAG': 'N'},
  'avg_disc': None,
  'avg_price': 35588.50968390804,
  'count_order': 348,
  'sum_base_price': 12384801.37,
  'sum_charge': 12282485.056933,
  'sum_disc_price': 11798257

## TPCH Q 2

In [11]:
pipeline2 = [{
        "$match":{
            "O_LINEITEMS.L_PARTSUPP.PS_PART.P_SIZE":17,
            "O_LINEITEMS.L_PARTSUPP.PS_PART.P_TYPE":{"$regex": 'STEEL', "$options": 'g'},
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_REGION.R_NAME":"ASIA",
        }
    },
   {
        "$project": {
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_ACCTBAL": 1,
           "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NAME":1,
           "O_LINEITEMS.L_PARTSUPP.S_NATION.N_NAME":1,
         
            "O_LINEITEMS.L_PARTSUPP.PS_PART.P_MFGR":1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_ADDRESS":1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_COMMENT":1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_PHONE":1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLYCOST":1,
        }
   },
   {
       "$sort":{
           "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_ACCTBAL":-1,
           "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NAME":1,
           "O_LINEITEMS.L_PARTSUPP.S_NATION.N_NAME":1,
      
       }
   }]

In [12]:
tick = time.time()
pprint.pprint(list(db.orders.aggregate(pipeline2)))

tock = time.time()

print("Total execution time: %.3f s" % (tock - tick))

[{'O_LINEITEMS': [{'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#5'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 5916.91,
                                                  'S_ADDRESS': 'fRJimA7zchyApqRLHcQeocVpP',
                                                  'S_COMMENT': 'into beans '
                                                               'wake after the '
                                                               'special '
                                                               'packages. '
                                                               'slyly fluffy '
                                                               'requests '
                                                               'cajole furio',
                                                  'S_NAME': 'Supplier#000000031',
                                                  'S_PHONE': '26-515-530-4159'},
                                  'PS_SUPPLYCOST': 582.85

                                                  'S_NAME': 'Supplier#000000041',
                                                  'S_PHONE': '28-739-447-2525'},
                                  'PS_SUPPLYCOST': 803.21}},
                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#4'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 3017.47,
                                                  'S_ADDRESS': 'cqjyB5h1nV',
                                                  'S_COMMENT': 'iously final '
                                                               'instructions. '
                                                               'quickly '
                                                               'special '
                                                               'accounts hang '
                                                               'fluffily above '
                                                               'the accounts. '
    

                                                  'S_ADDRESS': 'iybAE,RmTymrZVYaFZva2SH,j',
                                                  'S_COMMENT': 'n, ironic '
                                                               'ideas would '
                                                               'nag blithely '
                                                               'about the '
                                                               'slyly regular '
                                                               'accounts. '
                                                               'silent, expr',
                                                  'S_NAME': 'Supplier#000000020',
                                                  'S_PHONE': '13-715-945-6730'},
                                  'PS_SUPPLYCOST': 283.35}},
                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#2'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 35

                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#5'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': -290.06,
                                                  'S_ADDRESS': 'zyIeWzbbpkTV37vm1nmSGBxSgd2Kp',
                                                  'S_COMMENT': ' final '
                                                               'patterns. '
                                                               'accounts '
                                                               'haggle idly '
                                                               'pas',
                                                  'S_NAME': 'Supplier#000000040',
                                                  'S_PHONE': '32-231-247-6991'},
                                  'PS_SUPPLYCOST': 755.0}},
                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#1'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 9915.24,
               

                                                  'S_NAME': 'Supplier#000000060',
                                                  'S_PHONE': '18-550-360-2464'},
                                  'PS_SUPPLYCOST': 5.43}},
                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#2'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 5630.62,
                                                  'S_ADDRESS': 'jg0U '
                                                               'FNPMQDuyuKvTnLXXaLf3Wl6OtONA6mQlWJ',
                                                  'S_COMMENT': 'xpress '
                                                               'instructions '
                                                               'affix. '
                                                               'fluffily even '
                                                               'requests boos',
                                                  'S_NAME': 'Supplier#0000

...

                                                               'MBfH7',
                                                  'S_COMMENT': ' ideas poach '
                                                               'carefully '
                                                               'after the '
                                                               'blithely bold '
                                                               'asymptotes. '
                                                               'furiously '
                                                               'pending '
                                                               'theodoli',
                                                  'S_NAME': 'Supplier#000000026',
                                                  'S_PHONE': '31-758-894-4436'},
                                  'PS_SUPPLYCOST': 693.23}},
                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#1'},
               

                                  'PS_SUPPLYCOST': 269.77}},
                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#1'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 4515.87,
                                                  'S_ADDRESS': 'rGobqSMMYz0ErrPhCGS',
                                                  'S_COMMENT': 'warhorses. '
                                                               'ironic, '
                                                               'regular '
                                                               'accounts '
                                                               'detect slyly '
                                                               'after the '
                                                               'quickly f',
                                                  'S_NAME': 'Supplier#000000050',
                                                  'S_PHONE': '19-561-560-7437'},
            

                                                               'blith',
                                                  'S_NAME': 'Supplier#000000074',
                                                  'S_PHONE': '30-166-486-1559'},
                                  'PS_SUPPLYCOST': 916.1}},
                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#2'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 368.76,
                                                  'S_ADDRESS': 'wd1djjKXT,4zBm',
                                                  'S_COMMENT': 'yly final '
                                                               'accounts could '
                                                               'are carefully. '
                                                               'fluffily '
                                                               'ironic '
                                                               'instruct',
                 

 {'O_LINEITEMS': [{'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#3'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 530.82,
                                                  'S_ADDRESS': 'iybAE,RmTymrZVYaFZva2SH,j',
                                                  'S_COMMENT': 'n, ironic '
                                                               'ideas would '
                                                               'nag blithely '
                                                               'about the '
                                                               'slyly regular '
                                                               'accounts. '
                                                               'silent, expr',
                                                  'S_NAME': 'Supplier#000000020',
                                                  'S_PHONE': '13-715-945-6730'},
                                  'PS_SUPPLYCOST': 340.

                                                  'S_COMMENT': 'yly final '
                                                               'accounts could '
                                                               'are carefully. '
                                                               'fluffily '
                                                               'ironic '
                                                               'instruct',
                                                  'S_NAME': 'Supplier#000000093',
                                                  'S_PHONE': '26-528-528-1157'},
                                  'PS_SUPPLYCOST': 200.0}},
                  {'L_PARTSUPP': {'PS_PART': {'P_MFGR': 'Manufacturer#1'},
                                  'PS_SUPPLIER': {'S_ACCTBAL': 287.16,
                                                  'S_ADDRESS': 'WCk '
                                                               'XCHYzBA1dvJDSol4ZJQQcQN,',
         

## TPCH Q3

In [13]:
pipeline3 = [{  
      "$match":{  
         "O_CUSTOMER.C_MKTSEGMENT":"AUTOMOBILE",
         "O_ORDERDATE":{  
            "$lte":  "2000-01-01"
         },
         "O_LINEITEMS.L_SHIPDATE":{  
            "$gte": "1990-01-01"
         }
      }
   },
   {  
      "$unwind":"$O_LINEITEMS"
   },
   {  
      "$project":{  
         "O_ORDERKEY":1,
         "O_ORDERDATE":1,
         "O_SHIPPRIORITY":1,
         "O_LINEITEMS.L_EXTENDEDPRICE":1,
         "l_dis_min_1":{  
            "$subtract":[  
               1,
               "$O_LINEITEMS.L_DISCOUNT"
            ]
         }
      }
   },
   {  
      "$group":{  
         "_id":{  
            "O_ORDERKEY":"$O_ORDERKEY",
            "O_ORDERDATE":"$O_ORDERDATE",
            "O_SHIPPRIORITY":"$O_SHIPPRIORITY"
         },
         "revenue":{  
            "$sum":{  
               "$multiply":[  
                  "$O_LINEITEMS.L_EXTENDEDPRICE",
                  "$l_dis_min_1"
               ]
            }
         }
      }
   },
   {  
      "$sort":{  
         "revenue":1,
         "O_ORDERDATE":1
      }
   }]

In [14]:
tick = time.time()
pprint.pprint(list(db.orders.aggregate(pipeline3)))

tock = time.time()

print("Total execution time: %.3f s" % (tock - tick))

[{'_id': {'O_ORDERDATE': '1993-03-05', 'O_SHIPPRIORITY': 0},
  'revenue': 1807.4238},
 {'_id': {'O_ORDERDATE': '1997-08-21', 'O_SHIPPRIORITY': 0},
  'revenue': 1889.9008},
 {'_id': {'O_ORDERDATE': '1992-03-28', 'O_SHIPPRIORITY': 0},
  'revenue': 2022.9748000000002},
 {'_id': {'O_ORDERDATE': '1992-02-28', 'O_SHIPPRIORITY': 0},
  'revenue': 2447.922},
 {'_id': {'O_ORDERDATE': '1995-11-18', 'O_SHIPPRIORITY': 0},
  'revenue': 2998.0172},
 {'_id': {'O_ORDERDATE': '1993-06-12', 'O_SHIPPRIORITY': 0},
  'revenue': 3227.7009999999996},
 {'_id': {'O_ORDERDATE': '1997-10-29', 'O_SHIPPRIORITY': 0},
  'revenue': 3306.7124999999996},
 {'_id': {'O_ORDERDATE': '1994-01-19', 'O_SHIPPRIORITY': 0},
  'revenue': 4709.08},
 {'_id': {'O_ORDERDATE': '1993-08-01', 'O_SHIPPRIORITY': 0},
  'revenue': 5455.013999999999},
 {'_id': {'O_ORDERDATE': '1997-10-28', 'O_SHIPPRIORITY': 0},
  'revenue': 6321.5408},
 {'_id': {'O_ORDERDATE': '1994-06-18', 'O_SHIPPRIORITY': 0},
  'revenue': 6341.195699999999},
 {'_id': {'O_O

  'revenue': 94996.45559999999},
 {'_id': {'O_ORDERDATE': '1995-02-13', 'O_SHIPPRIORITY': 0},
  'revenue': 95317.32770000001},
 {'_id': {'O_ORDERDATE': '1994-03-08', 'O_SHIPPRIORITY': 0},
  'revenue': 95475.46},
 {'_id': {'O_ORDERDATE': '1993-09-12', 'O_SHIPPRIORITY': 0},
  'revenue': 95627.00140000001},
 {'_id': {'O_ORDERDATE': '1995-09-18', 'O_SHIPPRIORITY': 0},
  'revenue': 95981.14050000001},
 {'_id': {'O_ORDERDATE': '1992-09-07', 'O_SHIPPRIORITY': 0},
  'revenue': 96562.70079999999},
 {'_id': {'O_ORDERDATE': '1996-05-30', 'O_SHIPPRIORITY': 0},
  'revenue': 96729.93},
 {'_id': {'O_ORDERDATE': '1992-11-21', 'O_SHIPPRIORITY': 0},
  'revenue': 97014.7626},
 {'_id': {'O_ORDERDATE': '1997-08-11', 'O_SHIPPRIORITY': 0},
  'revenue': 97236.6538},
 {'_id': {'O_ORDERDATE': '1997-04-15', 'O_SHIPPRIORITY': 0},
  'revenue': 97388.2108},
 {'_id': {'O_ORDERDATE': '1995-12-12', 'O_SHIPPRIORITY': 0},
  'revenue': 97608.624},
 {'_id': {'O_ORDERDATE': '1997-11-02', 'O_SHIPPRIORITY': 0},
  'revenue': 

  'revenue': 149256.2648},
 {'_id': {'O_ORDERDATE': '1996-09-20', 'O_SHIPPRIORITY': 0},
  'revenue': 149468.106},
 {'_id': {'O_ORDERDATE': '1995-04-22', 'O_SHIPPRIORITY': 0},
  'revenue': 149638.9093},
 {'_id': {'O_ORDERDATE': '1997-10-18', 'O_SHIPPRIORITY': 0},
  'revenue': 149806.5534},
 {'_id': {'O_ORDERDATE': '1998-06-28', 'O_SHIPPRIORITY': 0},
  'revenue': 150546.198},
 {'_id': {'O_ORDERDATE': '1992-04-14', 'O_SHIPPRIORITY': 0},
  'revenue': 150612.1795},
 {'_id': {'O_ORDERDATE': '1995-01-31', 'O_SHIPPRIORITY': 0},
  'revenue': 150616.1347},
 {'_id': {'O_ORDERDATE': '1994-06-24', 'O_SHIPPRIORITY': 0},
  'revenue': 150659.3803},
 {'_id': {'O_ORDERDATE': '1996-09-18', 'O_SHIPPRIORITY': 0},
  'revenue': 150804.2655},
 {'_id': {'O_ORDERDATE': '1997-12-03', 'O_SHIPPRIORITY': 0},
  'revenue': 151136.978},
 {'_id': {'O_ORDERDATE': '1995-01-14', 'O_SHIPPRIORITY': 0},
  'revenue': 151424.3745},
 {'_id': {'O_ORDERDATE': '1995-09-02', 'O_SHIPPRIORITY': 0},
  'revenue': 151760.0937},
 {'_id':

  'revenue': 198905.83359999998},
 {'_id': {'O_ORDERDATE': '1992-04-02', 'O_SHIPPRIORITY': 0},
  'revenue': 199075.4719},
 {'_id': {'O_ORDERDATE': '1992-01-11', 'O_SHIPPRIORITY': 0},
  'revenue': 199189.4738},
 {'_id': {'O_ORDERDATE': '1992-06-20', 'O_SHIPPRIORITY': 0},
  'revenue': 199709.184},
 {'_id': {'O_ORDERDATE': '1994-11-05', 'O_SHIPPRIORITY': 0},
  'revenue': 199831.2596},
 {'_id': {'O_ORDERDATE': '1992-10-24', 'O_SHIPPRIORITY': 0},
  'revenue': 199838.4408},
 {'_id': {'O_ORDERDATE': '1994-07-08', 'O_SHIPPRIORITY': 0},
  'revenue': 199983.38919999998},
 {'_id': {'O_ORDERDATE': '1997-11-09', 'O_SHIPPRIORITY': 0},
  'revenue': 200074.071},
 {'_id': {'O_ORDERDATE': '1998-05-31', 'O_SHIPPRIORITY': 0},
  'revenue': 200740.1046},
 {'_id': {'O_ORDERDATE': '1997-01-06', 'O_SHIPPRIORITY': 0},
  'revenue': 200983.5032},
 {'_id': {'O_ORDERDATE': '1994-12-13', 'O_SHIPPRIORITY': 0},
  'revenue': 201140.31639999998},
 {'_id': {'O_ORDERDATE': '1995-11-26', 'O_SHIPPRIORITY': 0},
  'revenue': 

 {'_id': {'O_ORDERDATE': '1993-09-08', 'O_SHIPPRIORITY': 0},
  'revenue': 250440.06559999997},
 {'_id': {'O_ORDERDATE': '1997-01-31', 'O_SHIPPRIORITY': 0},
  'revenue': 251241.5665},
 {'_id': {'O_ORDERDATE': '1995-04-16', 'O_SHIPPRIORITY': 0},
  'revenue': 251473.42479999998},
 {'_id': {'O_ORDERDATE': '1992-02-13', 'O_SHIPPRIORITY': 0},
  'revenue': 251523.063},
 {'_id': {'O_ORDERDATE': '1994-02-24', 'O_SHIPPRIORITY': 0},
  'revenue': 251953.5171},
 {'_id': {'O_ORDERDATE': '1993-11-29', 'O_SHIPPRIORITY': 0},
  'revenue': 252325.7568},
 {'_id': {'O_ORDERDATE': '1993-01-12', 'O_SHIPPRIORITY': 0},
  'revenue': 252572.5807},
 {'_id': {'O_ORDERDATE': '1998-04-01', 'O_SHIPPRIORITY': 0},
  'revenue': 252623.1838},
 {'_id': {'O_ORDERDATE': '1992-09-17', 'O_SHIPPRIORITY': 0},
  'revenue': 253522.9573},
 {'_id': {'O_ORDERDATE': '1994-02-06', 'O_SHIPPRIORITY': 0},
  'revenue': 253712.26619999998},
 {'_id': {'O_ORDERDATE': '1995-02-12', 'O_SHIPPRIORITY': 0},
  'revenue': 253738.0429},
 {'_id': {'O

 {'_id': {'O_ORDERDATE': '1992-01-24', 'O_SHIPPRIORITY': 0},
  'revenue': 391222.58139999997},
 {'_id': {'O_ORDERDATE': '1997-11-12', 'O_SHIPPRIORITY': 0},
  'revenue': 391331.7785},
 {'_id': {'O_ORDERDATE': '1996-01-20', 'O_SHIPPRIORITY': 0},
  'revenue': 391823.77650000004},
 {'_id': {'O_ORDERDATE': '1992-10-05', 'O_SHIPPRIORITY': 0},
  'revenue': 393220.38769999996},
 {'_id': {'O_ORDERDATE': '1992-10-01', 'O_SHIPPRIORITY': 0},
  'revenue': 393443.4069},
 {'_id': {'O_ORDERDATE': '1995-05-06', 'O_SHIPPRIORITY': 0},
  'revenue': 394893.3003},
 {'_id': {'O_ORDERDATE': '1996-03-19', 'O_SHIPPRIORITY': 0},
  'revenue': 394987.2101},
 {'_id': {'O_ORDERDATE': '1994-12-11', 'O_SHIPPRIORITY': 0},
  'revenue': 395100.5852},
 {'_id': {'O_ORDERDATE': '1995-05-01', 'O_SHIPPRIORITY': 0},
  'revenue': 395434.69299999997},
 {'_id': {'O_ORDERDATE': '1995-12-18', 'O_SHIPPRIORITY': 0},
  'revenue': 398534.3883},
 {'_id': {'O_ORDERDATE': '1993-08-25', 'O_SHIPPRIORITY': 0},
  'revenue': 398558.4496},
 {'_

## TPCH query no 4

In [5]:

pipeline4 = [  
   {  
      "$project":{  
         "O_ORDERDATE":1,
         "O_ORDERPRIORITY":1,
         "eq":{  
            "$cond":[  
               {  
                  "$lt":[  
                     "$Items.L_COMMITDATE",
                     "$Items.L_RECEIPTDATE"
                  ]
               },
               0,
               1
            ]
         }
      }
   },
   
   {  
      "$match":{  
         "O_ORDERDATE":{  
            "$gte": "1993-07-01"
         },
         "O_ORDERDATE":{  
            "$lt": "1993-10-01"
         },
         "eq":{  
            "$eq":1
         }
      }
   },
   {  
      "$group":{  
         "_id":{  
            "O_ORDERPRIORITY":"$O_ORDERPRIORITY"
         },
         "order_count":{  
            "$sum":1
         }
      }
   },
   {  
      "$sort":{  
         "O_ORDERPRIORITY":1
      }
   }
]

In [6]:


tick = time.time()
pprint.pprint(list(db.orders.aggregate(pipeline4)))

tock = time.time()

print("Total execution time: %.3f s" % (tock - tick))

[{'_id': {'O_ORDERPRIORITY': '1-URGENT'}, 'order_count': 792},
 {'_id': {'O_ORDERPRIORITY': '2-HIGH'}, 'order_count': 798},
 {'_id': {'O_ORDERPRIORITY': '3-MEDIUM'}, 'order_count': 768},
 {'_id': {'O_ORDERPRIORITY': '5-LOW'}, 'order_count': 815},
 {'_id': {'O_ORDERPRIORITY': '4-NOT SPECIFIED'}, 'order_count': 779}]
Total execution time: 0.026 s


## TPCH Q6

In [15]:
pipeline6 = [{
        "$match":{
            "O_LINEITEMS.L_SHIPDATE":{
                "$gte": "1994-01-01",
                "$lt": "1995-01-01"
            },
            "O_LINEITEMS.L_QUANTITY":{
                "$lt": 24
            },
            "O_LINEITEMS.L_DISCOUNT":{
                "$gte": 0.06 - 0.01,
                "$lte": 0.06 + 0.01
            }
        }
    },
    {  
         "$unwind":"$O_LINEITEMS"
   },
    {
        "$project":{
            "O_LINEITEMS.L_EXTENDEDPRICE":1,
            "O_LINEITEMS.L_DISCOUNT":1
        }
    },
    {
        "$group": {
            "_id": {
                
            },
            "revenue":{
                "$sum":{
                    "$multiply": [
                        "$O_LINEITEMS.L_EXTENDEDPRICE",
                        "$O_LINEITEMS.L_DISCOUNT"
                    ]
                }
            }
        }
    }]

In [16]:
tick = time.time()
pprint.pprint(list(db.orders.aggregate(pipeline6)))

tock = time.time()

print("Total execution time: %.3f s" % (tock - tick))

[{'_id': {}, 'revenue': 17333410.1111}]
Total execution time: 0.084 s


## TPCH Q 7

In [17]:
pipeline7 = [{
        "$match": {
            "O_LINEITEMS.L_SHIPDATE":{
                "$gte": "1995-01-01",
                "$lte": "1996-12-31"
             },
            "$or": [{
                    "$and":[{
                        "O_CUSTOMER.C_NATION.N_NAME": "JAPAN"
                    }, {
                        "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME": "INDIA"
                    }]
            }, {
                "$and": [{
                    "O_CUSTOMER.C_NATION.N_NAME": "INDIA"
                    }, {
                        "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME": "JAPAN"
                }]
            }]
         }
    },
    {  
     "$unwind":"$O_LINEITEMS"
   },
    {
        "$project":{
            "O_LINEITEMS.L_EXTENDEDPRICE":1,
            "O_CUSTOMER.C_NATION.N_NAME": 1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME": 1,
            "O_LINEITEMS.L_SHIPDATE":1,
            "O_LINEITEMS.L_DISCOUNT":1,
            "l_dis_min_1":{  
                "$subtract":[  
                    1,
                    "$O_LINEITEMS.L_DISCOUNT"
                ]
         }
        }
    },
       {
        "$project":{
            "O_LINEITEMS.L_EXTENDEDPRICE":1,
            "O_CUSTOMER.C_NATION.N_NAME": 1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME": 1,
            "O_LINEITEMS.L_SHIPDATE":1,
            "O_LINEITEMS.L_DISCOUNT":1,
            "l_dis_min_1":{  
                "$subtract":[  
                    1,
                    "$O_LINEITEMS.L_DISCOUNT"
                ]
         },
            "volume":{
                "$multiply":[
                    "$O_LINEITEMS.L_EXTENDEDPRICE",
                    "$l_dis_min_1"
                ]
            }
        }
    },
    {
        "$group":{
            "_id":{
                "SUPP_NATION": "$O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME",
                "CUST_NATION": "$O_CUSTOMER.C_NATION.N_NAME",
                "L_SHIPDATE": "$O_LINEITEMS.L_SHIPDATE"
            },
            "revenue":{
                "$sum": "$volume"
            }
        }
    }]

In [18]:
tick = time.time()
pprint.pprint(list(db.orders.aggregate(pipeline7)))

tock = time.time()

print("Total execution time: %.3f s" % (tock - tick))

[{'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1997-01-17',
          'SUPP_NATION': 'ARGENTINA'},
  'revenue': 20382.5622},
 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1996-05-21',
          'SUPP_NATION': 'INDONESIA'},
  'revenue': 47806.401600000005},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1995-04-08',
          'SUPP_NATION': 'UNITED STATES'},
  'revenue': 59947.47},
 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1995-11-29',
          'SUPP_NATION': 'JAPAN'},
  'revenue': 14043.186},
 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1995-06-13',
          'SUPP_NATION': 'CHINA'},
  'revenue': 21664.559999999998},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1995-01-24',
          'SUPP_NATION': 'CANADA'},
  'revenue': 1686.2956000000001},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1996-07-30',
          'SUPP_NATION': 'INDONESIA'},
  'revenue': 31249.088000000003},
 {'_id': {'CUST_NAT

  'revenue': 42853.002},
 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1995-04-30',
          'SUPP_NATION': 'BRAZIL'},
  'revenue': 66671.1},
 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1995-09-20',
          'SUPP_NATION': 'JAPAN'},
  'revenue': 73932.70499999999},
 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1995-03-22',
          'SUPP_NATION': 'UNITED KINGDOM'},
  'revenue': 62777.915},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1995-03-07',
          'SUPP_NATION': 'VIETNAM'},
  'revenue': 37393.428799999994},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1995-02-04',
          'SUPP_NATION': 'INDIA'},
  'revenue': 33116.078499999996},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1995-11-18',
          'SUPP_NATION': 'INDIA'},
  'revenue': 20617.245600000002},
 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1994-11-14',
          'SUPP_NATION': 'JAPAN'},
  'revenue': 36906.303},
 {'_id

 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1996-03-21',
          'SUPP_NATION': 'KENYA'},
  'revenue': 108944.155},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1996-07-16',
          'SUPP_NATION': 'MOZAMBIQUE'},
  'revenue': 15987.367199999999},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1995-07-01',
          'SUPP_NATION': 'KENYA'},
  'revenue': 58391.1195},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1994-12-21',
          'SUPP_NATION': 'UNITED KINGDOM'},
  'revenue': 7269.6078},
 {'_id': {'CUST_NATION': 'INDIA',
          'L_SHIPDATE': '1995-01-14',
          'SUPP_NATION': 'INDONESIA'},
  'revenue': 58269.44},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1995-05-11',
          'SUPP_NATION': 'INDONESIA'},
  'revenue': 23401.68},
 {'_id': {'CUST_NATION': 'JAPAN',
          'L_SHIPDATE': '1996-03-31',
          'SUPP_NATION': 'EGYPT'},
  'revenue': 28990.7392},
 {'_id': {'CUST_NATION': 'JAPAN',
          

In [19]:
pipeline9 = [ {
        "$match":{
            "O_LINEITEMS.L_PARTSUPP.PS_PART.P_NAME":{
                "$regex": 'YELLOW$',
                "$options": 'i'
            }
        }
    },
    {  
     "$unwind":"$O_LINEITEMS"
   },
    {
        "$project": {
            "O_ORDERDATE": 1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME":1,
            "O_LINEITEMS.L_EXTENDEDPRICE": 1,
            "O_LINEITEMS.L_DISCOUNT": 1,
            "O_LINEITEMS.L_QUANTITY": 1,
            "l_dis_min_1": {
                "$subtract": [
                    1,
                    "$O_LINEITEMS.L_DISCOUNT"
                ]
            },
            "amount_2":{
                "$multiply": [
                    "$O_LINEITEMS.L_PARTSUPP.PS_SUPPLYCOST",
                    "$O_LINEITEMS.L_QUANTITY"
                ]
            }
        }
    },
    {
        "$project": {
            "O_ORDERDATE": 1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME":1,
            "l_dis_min_1": 1,
            "O_LINEITEMS.L_EXTENDEDPRICE": 1,
            "amount_2": 1,
            "amount_1": {
                "$multiply": [
                    "$l_dis_min_1",
                    "$O_LINEITEMS.L_EXTENDEDPRICE"
                ]
            }
        }
    },
    {
        "$project": {
            "O_ORDERDATE": 1,
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME":1,
            "amount_final": {
                "$subtract": [
                    "$amount_1",
                    "$amount_2"
                ]
            }
        }
    },
    {
        "$group": {
            "_id": {
                "O_YEAR": "$O_ORDERDATE",
                "NATION": "$O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME"
            },
            "sum_total": {
                "$sum": "$amount_final"
            }
        }
    },
    {
        "$sort": {
            "O_LINEITEMS.L_PARTSUPP.PS_SUPPLIER.S_NATION.N_NAME": 1,
            "O_ORDERDATE": 1
        }
    }]

In [20]:
tick = time.time()
pprint.pprint(list(db.orders.aggregate(pipeline9)))

tock = time.time()

print("Total execution time: %.3f s" % (tock - tick))

[{'_id': {'NATION': 'KENYA', 'O_YEAR': '1995-06-11'},
  'sum_total': 78883.29999999999},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1998-07-01'},
  'sum_total': 22964.144999999997},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1992-04-14'},
  'sum_total': 3990.0068999999985},
 {'_id': {'NATION': 'VIETNAM', 'O_YEAR': '1994-08-15'},
  'sum_total': 21531.978000000003},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1994-01-30'}, 'sum_total': 46896.9252},
 {'_id': {'NATION': 'JORDAN', 'O_YEAR': '1997-03-15'}, 'sum_total': 5058.9495},
 {'_id': {'NATION': 'RUSSIA', 'O_YEAR': '1994-04-02'},
  'sum_total': 24637.728000000003},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1995-09-28'}, 'sum_total': 37354.752},
 {'_id': {'NATION': 'BRAZIL', 'O_YEAR': '1995-05-08'}, 'sum_total': 34824.5632},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1996-05-12'},
  'sum_total': 16421.399999999998},
 {'_id': {'NATION': 'IRAN', 'O_YEAR': '1996-01-05'},
  'sum_total': 52072.90900000001},
 {'_id': {'NATION': 'INDIA', 'O_YEAR': '19

  'sum_total': 9791.032500000001},
 {'_id': {'NATION': 'JAPAN', 'O_YEAR': '1993-10-12'}, 'sum_total': 13348.1886},
 {'_id': {'NATION': 'ROMANIA', 'O_YEAR': '1994-03-19'},
  'sum_total': 31581.196799999998},
 {'_id': {'NATION': 'FRANCE', 'O_YEAR': '1997-02-08'},
  'sum_total': 41240.63679999999},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1993-07-22'}, 'sum_total': 67952.7648},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1995-11-09'},
  'sum_total': 31213.550000000007},
 {'_id': {'NATION': 'ARGENTINA', 'O_YEAR': '1992-04-14'},
  'sum_total': 73619.1092},
 {'_id': {'NATION': 'CHINA', 'O_YEAR': '1992-09-19'},
  'sum_total': 36039.14729999999},
 {'_id': {'NATION': 'SAUDI ARABIA', 'O_YEAR': '1993-08-08'},
  'sum_total': 33349.4066},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1992-05-01'},
  'sum_total': 21192.435599999997},
 {'_id': {'NATION': 'BRAZIL', 'O_YEAR': '1994-04-10'},
  'sum_total': 24266.088000000003},
 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1996-01-08'},
  'sum_total': 201

 {'_id': {'NATION': 'MOROCCO', 'O_YEAR': '1996-06-23'}, 'sum_total': 35322.32},
 {'_id': {'NATION': 'CHINA', 'O_YEAR': '1992-10-31'}, 'sum_total': 11582.7968},
 {'_id': {'NATION': 'ROMANIA', 'O_YEAR': '1996-10-27'}, 'sum_total': 43567.452},
 {'_id': {'NATION': 'ARGENTINA', 'O_YEAR': '1994-01-16'},
  'sum_total': 10199.867999999999},
 {'_id': {'NATION': 'BRAZIL', 'O_YEAR': '1993-12-27'}, 'sum_total': 35386.61},
 {'_id': {'NATION': 'VIETNAM', 'O_YEAR': '1994-05-07'},
  'sum_total': 9865.348999999998},
 {'_id': {'NATION': 'JAPAN', 'O_YEAR': '1997-02-20'},
  'sum_total': 72762.20169999999},
 {'_id': {'NATION': 'IRAN', 'O_YEAR': '1993-10-27'}, 'sum_total': 22865.8668},
 {'_id': {'NATION': 'VIETNAM', 'O_YEAR': '1995-12-31'},
  'sum_total': 107819.408},
 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1996-07-28'},
  'sum_total': 19231.968},
 {'_id': {'NATION': 'PERU', 'O_YEAR': '1992-02-17'},
  'sum_total': 6261.829999999998},
 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1993-07-23'},
  's

 {'_id': {'NATION': 'ETHIOPIA', 'O_YEAR': '1996-09-06'},
  'sum_total': 26781.192000000003},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1995-07-15'},
  'sum_total': 16954.563199999997},
 {'_id': {'NATION': 'MOZAMBIQUE', 'O_YEAR': '1998-01-15'},
  'sum_total': 23493.82},
 {'_id': {'NATION': 'INDIA', 'O_YEAR': '1998-05-06'}, 'sum_total': 43103.52},
 {'_id': {'NATION': 'GERMANY', 'O_YEAR': '1997-09-05'},
  'sum_total': 17452.473899999997},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1994-02-12'},
  'sum_total': 82473.67199999999},
 {'_id': {'NATION': 'MOZAMBIQUE', 'O_YEAR': '1993-09-13'},
  'sum_total': 1559.66},
 {'_id': {'NATION': 'ETHIOPIA', 'O_YEAR': '1997-09-15'},
  'sum_total': 15305.258000000002},
 {'_id': {'NATION': 'GERMANY', 'O_YEAR': '1997-07-15'},
  'sum_total': 14896.952200000003},
 {'_id': {'NATION': 'RUSSIA', 'O_YEAR': '1996-06-14'}, 'sum_total': 30340.22},
 {'_id': {'NATION': 'RUSSIA', 'O_YEAR': '1996-03-30'}, 'sum_total': 46009.6104},
 {'_id': {'NATION': 'BRAZIL', 'O_YEAR': 

  'sum_total': 31951.304000000004},
 {'_id': {'NATION': 'FRANCE', 'O_YEAR': '1993-11-10'}, 'sum_total': 7730.6465},
 {'_id': {'NATION': 'CHINA', 'O_YEAR': '1994-05-22'}, 'sum_total': 28235.55},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1998-05-11'},
  'sum_total': 24142.5144},
 {'_id': {'NATION': 'RUSSIA', 'O_YEAR': '1995-07-16'}, 'sum_total': 43600.6528},
 {'_id': {'NATION': 'CANADA', 'O_YEAR': '1993-05-24'},
  'sum_total': 3988.3103999999985},
 {'_id': {'NATION': 'VIETNAM', 'O_YEAR': '1993-09-13'}, 'sum_total': 46007.276},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1997-05-17'},
  'sum_total': 11669.630400000002},
 {'_id': {'NATION': 'EGYPT', 'O_YEAR': '1998-04-25'},
  'sum_total': 17289.753600000004},
 {'_id': {'NATION': 'EGYPT', 'O_YEAR': '1993-02-08'},
  'sum_total': 51673.234200000006},
 {'_id': {'NATION': 'FRANCE', 'O_YEAR': '1992-12-01'}, 'sum_total': 7996.9316},
 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1998-05-11'},
  'sum_total': 56751.9176},
 {'_id': {'NATION': 'EG

...

  'sum_total': 57576.514500000005},
 {'_id': {'NATION': 'IRAQ', 'O_YEAR': '1993-10-10'},
  'sum_total': 35623.920000000006},
 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1996-09-12'},
  'sum_total': 19181.5585},
 {'_id': {'NATION': 'UNITED KINGDOM', 'O_YEAR': '1996-01-16'},
  'sum_total': 13714.182000000004},
 {'_id': {'NATION': 'IRAN', 'O_YEAR': '1997-05-31'},
  'sum_total': 5968.207999999999},
 {'_id': {'NATION': 'IRAN', 'O_YEAR': '1993-10-05'},
  'sum_total': 28259.905300000002},
 {'_id': {'NATION': 'MOROCCO', 'O_YEAR': '1992-02-22'},
  'sum_total': 57882.19200000001},
 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1992-06-26'},
  'sum_total': 65539.12},
 {'_id': {'NATION': 'INDIA', 'O_YEAR': '1996-12-13'},
  'sum_total': 30910.568000000003},
 {'_id': {'NATION': 'IRAQ', 'O_YEAR': '1996-11-07'}, 'sum_total': 55511.0808},
 {'_id': {'NATION': 'MOROCCO', 'O_YEAR': '1995-04-16'},
  'sum_total': 14674.976799999997},
 {'_id': {'NATION': 'FRANCE', 'O_YEAR': '1993-12-27'}, 'sum_total': 3

  'sum_total': 17762.64879999999},
 {'_id': {'NATION': 'IRAN', 'O_YEAR': '1992-09-03'},
  'sum_total': 11325.250999999997},
 {'_id': {'NATION': 'RUSSIA', 'O_YEAR': '1997-09-05'},
  'sum_total': 25766.726000000002},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1992-01-29'}, 'sum_total': 44720.91},
 {'_id': {'NATION': 'IRAN', 'O_YEAR': '1996-11-06'}, 'sum_total': 16370.4057},
 {'_id': {'NATION': 'JAPAN', 'O_YEAR': '1996-06-09'},
  'sum_total': 1737.322400000001},
 {'_id': {'NATION': 'GERMANY', 'O_YEAR': '1996-09-14'}, 'sum_total': 37480.347},
 {'_id': {'NATION': 'ALGERIA', 'O_YEAR': '1994-03-03'},
  'sum_total': 1248.3611999999994},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1998-08-02'}, 'sum_total': 6487.2744},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1993-09-11'}, 'sum_total': 44143.8476},
 {'_id': {'NATION': 'JAPAN', 'O_YEAR': '1993-10-10'}, 'sum_total': 14189.4},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1992-08-15'},
  'sum_total': 9038.589999999997},
 {'_id': {'NATION': 'IRAQ', 'O_YEAR':

 {'_id': {'NATION': 'MOROCCO', 'O_YEAR': '1995-01-23'},
  'sum_total': 59427.92259999999},
 {'_id': {'NATION': 'ALGERIA', 'O_YEAR': '1994-12-25'},
  'sum_total': 15931.979000000001},
 {'_id': {'NATION': 'BRAZIL', 'O_YEAR': '1992-06-03'}, 'sum_total': 42692.8401},
 {'_id': {'NATION': 'JORDAN', 'O_YEAR': '1995-02-13'},
  'sum_total': 37436.226299999995},
 {'_id': {'NATION': 'EGYPT', 'O_YEAR': '1994-08-30'},
  'sum_total': 39729.537599999996},
 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1993-10-16'},
  'sum_total': 22660.092000000008},
 {'_id': {'NATION': 'UNITED KINGDOM', 'O_YEAR': '1996-07-01'},
  'sum_total': 18450.6434},
 {'_id': {'NATION': 'INDIA', 'O_YEAR': '1992-09-03'}, 'sum_total': 62594.094},
 {'_id': {'NATION': 'JAPAN', 'O_YEAR': '1997-07-21'}, 'sum_total': 26136.6588},
 {'_id': {'NATION': 'INDONESIA', 'O_YEAR': '1997-05-25'},
  'sum_total': 31595.329799999996},
 {'_id': {'NATION': 'VIETNAM', 'O_YEAR': '1993-12-11'}, 'sum_total': 38800.936},
 {'_id': {'NATION': 'INDIA', 'O_

 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1998-07-08'},
  'sum_total': 5400.737999999999},
 {'_id': {'NATION': 'KENYA', 'O_YEAR': '1992-03-19'}, 'sum_total': 27810.2848},
 {'_id': {'NATION': 'UNITED KINGDOM', 'O_YEAR': '1992-11-06'},
  'sum_total': 27221.303799999994},
 {'_id': {'NATION': 'ROMANIA', 'O_YEAR': '1994-03-23'},
  'sum_total': 7986.146000000001},
 {'_id': {'NATION': 'CHINA', 'O_YEAR': '1992-11-04'}, 'sum_total': 4125.1472},
 {'_id': {'NATION': 'UNITED KINGDOM', 'O_YEAR': '1994-06-12'},
  'sum_total': 46106.796},
 {'_id': {'NATION': 'UNITED KINGDOM', 'O_YEAR': '1992-08-19'},
  'sum_total': 4848.235499999999},
 {'_id': {'NATION': 'GERMANY', 'O_YEAR': '1995-12-04'},
  'sum_total': 17528.2839},
 {'_id': {'NATION': 'ROMANIA', 'O_YEAR': '1996-09-10'},
  'sum_total': 10323.697999999997},
 {'_id': {'NATION': 'ARGENTINA', 'O_YEAR': '1997-05-16'},
  'sum_total': 8457.670000000002},
 {'_id': {'NATION': 'CHINA', 'O_YEAR': '1992-07-10'},
  'sum_total': 11062.672400000001},
 {'_id'

  'sum_total': 14120.158299999996},
 {'_id': {'NATION': 'UNITED STATES', 'O_YEAR': '1992-08-04'},
  'sum_total': 49918.71409999999},
 {'_id': {'NATION': 'MOZAMBIQUE', 'O_YEAR': '1993-08-05'},
  'sum_total': 27350.817199999998},
 {'_id': {'NATION': 'RUSSIA', 'O_YEAR': '1993-05-11'}, 'sum_total': 14192.1052},
 {'_id': {'NATION': 'JAPAN', 'O_YEAR': '1993-05-06'},
  'sum_total': 4688.650800000001},
 {'_id': {'NATION': 'FRANCE', 'O_YEAR': '1996-08-01'},
  'sum_total': 4265.076000000001},
 {'_id': {'NATION': 'MOZAMBIQUE', 'O_YEAR': '1996-04-07'},
  'sum_total': 7793.411999999998},
 {'_id': {'NATION': 'ARGENTINA', 'O_YEAR': '1994-09-14'},
  'sum_total': 722.1191999999996},
 {'_id': {'NATION': 'EGYPT', 'O_YEAR': '1995-04-07'},
  'sum_total': 6013.8276000000005},
 {'_id': {'NATION': 'GERMANY', 'O_YEAR': '1994-08-30'}, 'sum_total': 57219.63},
 {'_id': {'NATION': 'CHINA', 'O_YEAR': '1997-06-11'},
  'sum_total': 523.0323999999982},
 {'_id': {'NATION': 'SAUDI ARABIA', 'O_YEAR': '1993-07-27'},
  'su